### Notes
Files:
  - blast_P07237.xml: Result of blast
  - hitted_seqs_from_blast_P07237.fasta: Full sequences of the proteins hit in blast
  - clustal_blast_P07237.fasta: Clustal Omega MSA of the hitted proteins
  - refined_clustal_blast_P07237.fasta: Same as previous but refined using JalView. The msa was refined in different ways and so there are different versions of this file. The techniques applyied for refining are described below.
  - irrelevant.fasta: Irrelevant file required by psiblast script

### Steps and Code

In [1]:
# !python -m pip install biopython
from Bio import SeqIO
from Bio.Blast import NCBIXML
import pandas as pd
import requests

1. Perform a blast search using our assigned sequence on https://www.ebi.ac.uk/Tools/sss/ncbiblast \
Parameters:
  - database: UniProtKB/Swiss-Prot
  - alignment-views: BLASTXML
  - exp. thr.: 10
  - scores: 500
  - alignments: 500

2. Download results doing: "Tool output" --> Right click "Download" --> "Save link as..." --> "blast_P07237.xml"


3. Format hits in a dataframe

In [5]:
blastFile = 'blast_P07237.xml'

In [6]:
with open(blastFile) as f:
    blast_records = NCBIXML.parse(f)
    data = []

    # Iterate PSIBLAST rounds (here just one since it is a simple BLAST)
    for blast_record in blast_records:
        query_id = blast_record.query
        # Iterate queries (here just one)
        for i, alignment in enumerate(blast_record.alignments):
            subject_id = alignment.title
            # Iterate pairwise alignments
            for hsp in alignment.hsps:
                data.append((query_id,
                              subject_id,
                              blast_record.query_length,
                              hsp.query,
                              hsp.match,
                              hsp.sbjct,
                              hsp.query_start,
                              hsp.query_end,
                              hsp.sbjct_start,
                              hsp.sbjct_end,
                              hsp.identities,
                              hsp.positives,
                              hsp.gaps,
                              hsp.expect,
                              hsp.score))
                # Skip duplicated subjects
                break
                
df = pd.DataFrame(data, columns=["query_id", "subject_id", "query_len",
                                  "query_seq", "match_seq", "subject_seq",
                                "query_start", "query_end", "subject_start", "subject_end", 
                                "identity", "positive", "gaps", "eval", "bit_score"])

display(df.head(2))
print(df.shape)

,query_id,subject_id,query_len,query_seq,match_seq,subject_seq,query_start,query_end,subject_start,subject_end,identity,positive,gaps,eval,bit_score
0,EMBOSS_001,SP:Q5R5B6 PDIA1_PONAB Protein disulfide-isomer...,106,VLVLRKSNFAEALAAHKYLLVEFYAPWCGHCKALAPEYAKAAGKLK...,VLVLRKSNFAEALAAHKYLLVEFYAPWCGHCKALAPEYAKAAGKLK...,VLVLRKSNFAEALAAHKYLLVEFYAPWCGHCKALAPEYAKAAGKLK...,1,106,26,131,106,106,0,2.440510e-69,555.0
1,EMBOSS_001,SP:P07237 PDIA1_HUMAN Protein disulfide-isomer...,106,VLVLRKSNFAEALAAHKYLLVEFYAPWCGHCKALAPEYAKAAGKLK...,VLVLRKSNFAEALAAHKYLLVEFYAPWCGHCKALAPEYAKAAGKLK...,VLVLRKSNFAEALAAHKYLLVEFYAPWCGHCKALAPEYAKAAGKLK...,1,106,26,131,106,106,0,2.440510e-69,555.0


(500, 15)


In [7]:
df.to_excel('temp.xlsx')

In [9]:
# Select just the subset of significan hits. I chose as threshold an e-value of 0.001
sigDf = df.iloc[df.index[df['eval'] < 0.001]]
print(sigDf.shape)

(362, 15)


4. Retrieve full protein sequences from the api https://www.ebi.ac.uk/proteins/api/proteins

In [10]:
# Download sequences in chunks (URL length is limited)

out_file = 'prot_seqs_from_blast_P07237.fasta'

URL = "https://www.ebi.ac.uk/proteins/api/proteins"

print(sigDf['subject_id'][0])

chunk_size = 100 
with open(out_file, 'w') as fout:
    accessions = list(set([acc.split(' ')[0].split(":")[1] for acc in sigDf['subject_id']]))
    for i in range(0, len(accessions), chunk_size):
        r = requests.get(URL, params={'accession': ','.join(accessions[i: i + chunk_size])}, 
                      headers={'Accept': 'text/x-fasta'})
        print(r.status_code)
        fout.write(r.text + "\n")
        



SP:Q5R5B6 PDIA1_PONAB Protein disulfide-isomerase OS=Pongo abelii OX=9601 GN=P4HB PE=2 SV=1
200
200
200
200


5. Generate MSA from Clustal Omega at https://www.ebi.ac.uk/Tools/msa/clustalo/ \
Parameters:
  - output format: Pearson/FASTA

6. Refine results with JalView. \
    I produced different refined alignments:
    1. (refined_1) Removed redundancy above 100% threshold, recomputed alignment with ClustalO with Default parameters, then removed empty columns, and then recomputed the alignment again. To do all this steps I just used the built-in functions in the upper toolbar.
    2. (refined_2) No refinement.
    3. (refined_3) Removed before 1256 and after 1457, then removed redundancy above 100% threshold, and then realigned with ClustalO with Default parameters.
    4. (refined_4) Selected just the most conserved region, removing up to 1314 included, and from 1337 included on. Then removed redundancy as before and realigned.
    5. (refined_5) Removed up to 328 included, and from 2599 included on. Then removed redundancy and realigned.

7. Generate a pssm model with command: \
`psiblast -subject irrelevant.fasta -in_msa refined_clustal_blast_P07237.fasta -out_ascii_pssm models/P07237_ascii_pssm.pssm_ascii -out_pssm models/P07237_pssm.pssm`

To install psiblast on linux use:

In [ ]:
!wget https://ftp.ncbi.nlm.nih.gov/blast/executables/blast+/LATEST/ncbi-blast-2.13.0+-x64-linux.tar.gz
!tar -xzf ncbi-blast-2.13.0+-x64-linux.tar.gz
# Test
!./ncbi-blast-2.13.0+/bin/psiblast

8. Generate a hmm model with command: \
`hmmbuild P07237_hmm.hmm refined_clustal_blast_P07237.fasta`

To install hmmbuild on linux use:

In [ ]:
!wget http://eddylab.org/software/hmmer/hmmer.tar.gz
!tar -xzf hmmer.tar.gz
!cd hmmer-3.3.2
!./configure
!make
# Test
!./src/hmmbuild

or more simply

In [ ]:
!sudo apt install hmmer